In [1]:
import os, sys
import numpy as np
import tensorflow as tf
from bilm import Batcher, TokenBatcher, BidirectionalLanguageModel, weight_layers, dump_token_embeddings
import com.capone.utils.data_utils as du

In [2]:
datadir = '/Users/yqk798/Downloads/datasets/elmo'
vocab_file = os.path.join(datadir, 'vocab.txt')
options_file = os.path.join(datadir, 'options.json')
weight_file = os.path.join(datadir, 'lm_weights.hdf5')

In [3]:
# Create a Batcher to map text to character ids.
batcher = Batcher(vocab_file, 50)
token_batcher = TokenBatcher(vocab_file)

# Input placeholders to the biLM.
question_character_ids = tf.placeholder('int32', shape=(None, None, 50))
question_token_ids = tf.placeholder('int32', shape=(None, None))

In [4]:
token_embedding_file = os.path.join(datadir, 'elmo_token_embeddings.hdf5')

In [ ]:
dump_token_embeddings(
    vocab_file, options_file, weight_file, token_embedding_file
)
tf.reset_default_graph()

Instructions for updating:
Colocations handled automatically by placer.


2018-11-30 16:23:53,037 From /Users/yqk798/Library/Python/3.6/lib/python/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use the `axis` argument instead


2018-11-30 16:23:53,082 From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/bilm-0.1.post5-py3.6.egg/bilm/model.py:384: calling squeeze (from tensorflow.python.ops.array_ops) with squeeze_dims is deprecated and will be removed in a future version.
Instructions for updating:
Use the `axis` argument instead


USING SKIP CONNECTIONS


In [5]:
# Build the biLM graph.
bilm = BidirectionalLanguageModel(options_file, weight_file)
bilm_token = BidirectionalLanguageModel(
    options_file,
    weight_file,
    use_character_inputs=False,
    embedding_weight_file=token_embedding_file
)

In [6]:
#question_embeddings_op = bilm(question_character_ids)
question_embeddings_op_token = bilm_token(question_token_ids)

Instructions for updating:
Colocations handled automatically by placer.


2018-12-03 11:49:25,378 From /Users/yqk798/Library/Python/3.6/lib/python/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


USING SKIP CONNECTIONS


In [7]:
with tf.variable_scope('', tf.AUTO_REUSE):
    # the reuse=True scope reuses weights from the context for the question
    elmo_question_input = weight_layers(
        'input', question_embeddings_op_token, l2_coef=0.0
    )

Instructions for updating:
Use the `axis` argument instead


2018-12-03 11:49:27,080 From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/bilm-0.1.post5-py3.6.egg/bilm/elmo.py:89: calling squeeze (from tensorflow.python.ops.array_ops) with squeeze_dims is deprecated and will be removed in a future version.
Instructions for updating:
Use the `axis` argument instead


In [8]:
tokenized_question = [
    ['what', 'are', 'biLMs', 'useful', 'for', '?', 'Varun'],
    ['sure', 'useful']
]

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [10]:
# It is necessary to initialize variables once before running inference.
question_ids = token_batcher.batch_sentences(tokenized_question)
print(question_ids)
elmo_question_input_ = sess.run(elmo_question_input['weighted_op'], 
                                feed_dict={question_token_ids: question_ids}
)
print(elmo_question_input_)
print(elmo_question_input_.shape)

[[     0    103     33 400002   4700     11    189 400002      0]
 [     0   1086   4700      0      0      0      0      0      0]]
[[[ 0.7411803  -0.18728203  0.9542081  ...  0.46182865 -0.66130084
    0.12696877]
  [-0.5487164  -0.10489723 -0.00882029 ...  0.4861088   0.05831876
    0.13532063]
  [-1.2277577  -2.3290477   3.3235774  ...  0.9437058  -2.3616962
    3.2876296 ]
  ...
  [-0.115685   -0.9261884  -0.6973815  ...  0.75920975 -0.6579843
   -0.251489  ]
  [-0.95735115 -1.3376005   2.327537   ...  0.8914468  -2.533528
    2.483564  ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[-0.05013221 -0.18088299  0.2289171  ...  0.13237412 -0.22771375
   -0.1142216 ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.


In [8]:
conll_train = du.CoNLLDataset('./data/ner/combined/train.txt')

In [9]:
training_sents = []
for (words, tags, _, _) in conll_train:
    training_sents.append(words)

In [10]:
len(training_sents)

74954

In [11]:
sent_lengths = [len(sent) for sent in training_sents]
max_sent_len = max(sent_lengths)
max_sent_len

63

In [29]:
BATCH_SIZE = 128
BATCHES = len(training_sents) // BATCH_SIZE + (len(training_sents) % BATCH_SIZE != 0)
print(BATCHES)

586


In [30]:
output_array = np.zeros(shape=[len(training_sents), max_sent_len, 512])

In [32]:
for batch in range(BATCHES):
    batch_data = training_sents[batch * BATCH_SIZE : (batch + 1) * BATCH_SIZE]
    question_ids = batcher.batch_sentences(batch_data)
    #question_ids = batcher.batch_sentences(training_sents[idx: idx+1])
    elmo_question_input_ = sess.run(elmo_question_input['weighted_op'], 
                                    feed_dict={question_character_ids: question_ids})
    shape = elmo_question_input_.shape
    output_array[batch * BATCH_SIZE : (batch + 1) * BATCH_SIZE, :shape[1], :shape[2]] = elmo_question_input_
    #if batch % 100 == 0:
    print("BATCH: %d" % batch)

BATCH: 0
BATCH: 1
BATCH: 2
BATCH: 3
BATCH: 4
BATCH: 5
BATCH: 6
BATCH: 7
BATCH: 8
BATCH: 9
BATCH: 10
BATCH: 11
BATCH: 12
BATCH: 13
BATCH: 14
BATCH: 15
BATCH: 16
BATCH: 17
BATCH: 18
BATCH: 19
BATCH: 20
BATCH: 21
BATCH: 22
BATCH: 23
BATCH: 24
BATCH: 25
BATCH: 26
BATCH: 27
BATCH: 28
BATCH: 29
BATCH: 30
BATCH: 31
BATCH: 32
BATCH: 33
BATCH: 34
BATCH: 35
BATCH: 36
BATCH: 37
BATCH: 38
BATCH: 39
BATCH: 40
BATCH: 41
BATCH: 42
BATCH: 43
BATCH: 44
BATCH: 45
BATCH: 46
BATCH: 47
BATCH: 48
BATCH: 49
BATCH: 50
BATCH: 51
BATCH: 52
BATCH: 53
BATCH: 54
BATCH: 55
BATCH: 56
BATCH: 57
BATCH: 58
BATCH: 59
BATCH: 60
BATCH: 61
BATCH: 62
BATCH: 63
BATCH: 64
BATCH: 65
BATCH: 66
BATCH: 67
BATCH: 68
BATCH: 69
BATCH: 70
BATCH: 71
BATCH: 72
BATCH: 73
BATCH: 74
BATCH: 75
BATCH: 76
BATCH: 77
BATCH: 78
BATCH: 79
BATCH: 80
BATCH: 81
BATCH: 82
BATCH: 83
BATCH: 84
BATCH: 85
BATCH: 86
BATCH: 87
BATCH: 88
BATCH: 89
BATCH: 90
BATCH: 91
BATCH: 92
BATCH: 93
BATCH: 94
BATCH: 95
BATCH: 96
BATCH: 97
BATCH: 98
BATCH: 99
BATCH: 100

In [ ]:
output_array[0][5]

In [39]:
output_array1 = np.copy(output_array)

In [40]:
for idx in range(len(training_sents)):
    output_array1[idx, len(training_sents[idx]):, :] = 0.0

In [37]:
np.save('/Users/yqk798/dev/notebooks/data/ner/combined/train-1.npy', output_array1)

In [41]:
from scipy import sparse
from scipy.sparse import coo_matrix

In [43]:
sparse_embeddings = sparse.coo_matrix(output_array1)

TypeError: expected dimension <= 2 array or matrix